In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam
import os
import sys

/home/sam/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:53: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


## Importation des données

In [47]:
%matplotlib inline
plt.style.use('seaborn-whitegrid') # theme
plt.rcParams['figure.figsize'] = (12,8) # taille des figures
package_dir = os.path.dirname(os.path.abspath(sys.argv[1]))
#file_name  = datetime.datetime(2018, 6, 1).strftime("%B")
file_name = "exctract"
thefile = os.path.join(package_dir, file_name+'.csv')
print(thefile)
raw_ext = pd.read_csv(thefile)
raw_ext = ext.sort_values(by="date")
print(raw_ext.shape)
ext = raw_ext.copy()
raw_ext.head(5)

/home/sam/Documents/football-ligue-1-scraping-eda/exctract.csv
(5972, 11)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b
5971,2004-02-14 16:15:00,Saturday,16:15,1.77,4.22,3.13,1,1,2003/2004,Sochaux,Bordeaux
5963,2004-02-14 19:00:00,Saturday,19:00,2.03,3.30,3.07,0,0,2003/2004,AC Ajaccio,Strasbourg
5964,2004-02-14 19:00:00,Saturday,19:00,2.00,3.40,3.07,1,0,2003/2004,Guingamp,Bastia
5965,2004-02-14 19:00:00,Saturday,19:00,2.98,2.18,3.05,0,0,2003/2004,Le Mans,Marseille
5970,2004-02-14 19:00:00,Saturday,19:00,3.48,1.97,3.05,0,1,2003/2004,Toulouse,Paris SG


## Résultat du match 

Création de la variable *res* 

In [57]:
ext = raw_ext.copy()

#Cration de res : resultat du match
def func_res(row):
    res = 'HOME'
    if(row.score_b > row.score_a):
        res = 'AWAY'
    elif(row.score_b == row.score_a) :
        res = 'DRAW'
    return res

ext['res'] = ext.apply(func_res, axis=1)
print(ext.shape)
ext.head(5)

(5972, 12)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b,res
5971,2004-02-14 16:15:00,Saturday,16:15,1.77,4.22,3.13,1,1,2003/2004,Sochaux,Bordeaux,DRAW
5963,2004-02-14 19:00:00,Saturday,19:00,2.03,3.30,3.07,0,0,2003/2004,AC Ajaccio,Strasbourg,DRAW
5964,2004-02-14 19:00:00,Saturday,19:00,2.00,3.40,3.07,1,0,2003/2004,Guingamp,Bastia,HOME
5965,2004-02-14 19:00:00,Saturday,19:00,2.98,2.18,3.05,0,0,2003/2004,Le Mans,Marseille,DRAW
5970,2004-02-14 19:00:00,Saturday,19:00,3.48,1.97,3.05,0,1,2003/2004,Toulouse,Paris SG,AWAY


## min_odds
Quelle est la plus petite cote attribué par le bookmaker ? 

In [4]:
# min_odds : qui le bookmaker voyait gagnant (plus petite des cotes)
def func_min_odds(row):
    # on prend l'index de la cote la plus petite : 
    min_index = [row.odds_a, row.odds_b, row.odds_draw].index(min([row.odds_a, row.odds_b, row.odds_draw]))
    # on la traduit en classe
    return ['HOME', 'AWAY', 'DRAW'][min_index]
    #prediction_odds = 1 if outcome_odds == res else 0 

ext['min_odds'] = ext.apply(func_min_odds, axis=1)
print(ext.shape)
ext.head()

(5972, 13)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b,res,min_odds
5971,2004-02-14 16:15:00,Saturday,16:15,1.77,4.22,3.13,1,1,2003/2004,Sochaux,Bordeaux,DRAW,HOME
5963,2004-02-14 19:00:00,Saturday,19:00,2.03,3.30,3.07,0,0,2003/2004,AC Ajaccio,Strasbourg,DRAW,HOME
5964,2004-02-14 19:00:00,Saturday,19:00,2.00,3.40,3.07,1,0,2003/2004,Guingamp,Bastia,HOME,HOME
5965,2004-02-14 19:00:00,Saturday,19:00,2.98,2.18,3.05,0,0,2003/2004,Le Mans,Marseille,DRAW,AWAY
5970,2004-02-14 19:00:00,Saturday,19:00,3.48,1.97,3.05,0,1,2003/2004,Toulouse,Paris SG,AWAY,AWAY


## max_odds 
Plus grande cote 
(Qui le bookmaker voyait perdant plus grande des cotes)


In [5]:
# max_odds : qui le bookmaker voyait perdant (plus grande des cotes)
def func_max_odds(row):
    # on prend l'index de la cote la plus petite : 
    max_index = [row.odds_a, row.odds_b, row.odds_draw].index(max([row.odds_a, row.odds_b, row.odds_draw]))
    # on la traduit en classe
    return ['HOME', 'AWAY', 'DRAW'][max_index]
ext['max_odds'] = ext.apply(func_max_odds, axis=1)
print(ext.shape)
ext.head()

(5972, 14)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b,res,min_odds,max_odds
5971,2004-02-14 16:15:00,Saturday,16:15,1.77,4.22,3.13,1,1,2003/2004,Sochaux,Bordeaux,DRAW,HOME,AWAY
5963,2004-02-14 19:00:00,Saturday,19:00,2.03,3.30,3.07,0,0,2003/2004,AC Ajaccio,Strasbourg,DRAW,HOME,AWAY
5964,2004-02-14 19:00:00,Saturday,19:00,2.00,3.40,3.07,1,0,2003/2004,Guingamp,Bastia,HOME,HOME,AWAY
5965,2004-02-14 19:00:00,Saturday,19:00,2.98,2.18,3.05,0,0,2003/2004,Le Mans,Marseille,DRAW,AWAY,DRAW
5970,2004-02-14 19:00:00,Saturday,19:00,3.48,1.97,3.05,0,1,2003/2004,Toulouse,Paris SG,AWAY,AWAY,HOME


## is_booky_write
Partons du principe que la cote la plus petite est celle qui represente celle qui a le plus de chance de passer. 
C'est le choix du bookmaker.


le bookmaker a raison si la cote la plus petite correspond au resultat du match 

In [6]:
# good_pred le bookmaker a prédit le resultat effectif
ext['is_booky_write'] =  pd.Series(ext['min_odds'] == ext['res'],dtype='int64')
# la plus grande cote l'emporte
ext['worst_pred'] =  pd.Series(ext['max_odds'] == ext['res'],dtype='int64')
ext['draw_pred'] = pd.Series((ext['is_booky_write']==0) & (ext['worst_pred'] == 0), dtype='int64')
print(ext.shape)
ext[ext.draw_pred == 1].head(2)

(5972, 17)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b,res,min_odds,max_odds,is_booky_write,worst_pred,draw_pred
5971,2004-02-14 16:15:00,Saturday,16:15,1.77,4.22,3.13,1,1,2003/2004,Sochaux,Bordeaux,DRAW,HOME,AWAY,0,0,1
5963,2004-02-14 19:00:00,Saturday,19:00,2.03,3.30,3.07,0,0,2003/2004,AC Ajaccio,Strasbourg,DRAW,HOME,AWAY,0,0,1


## Calculer le classement
### Comment est init le classement ? 
Ex : [Fifa](https://www.lefigaro.fr/le-scan-sport/le-saviez-vous/2014/07/17/27006-20140717ARTFIG00090-comment-est-calcule-le-classement-fifa.php), [Ligue 1](https://www.webchercheurs.com/34/1501-fr-calcul-points-classement-ligue-1-bareme-football.html)

## points_a et b :
- Une victoire vaut trois points
- Le match nul un point
- La défaite ne rapporte aucun point

Amélioration : 
    - Classement par saison 
    - Initialisation du classement


In [49]:
ext = raw_ext.copy()

In [7]:
# ajouter season, faire une fonction 
# modifier filter 
# attention ! ajouter ext['team_a'] == team_a
season = "2010/2011"
def func_points(ext, season):
    ext['points_a'] = ext.apply(lambda row : 3 if row.res == 'HOME' else (1  if row.res == 'DRAW' else 0), axis = 1)
    ext['points_b'] = ext.apply(lambda row : 3 if row.res == 'AWAY' else (1  if row.res == 'DRAW' else 0), axis = 1)
    
    # team_a
    for team_a in ext.team_a.unique():
        a_filter = (ext['team_a'] == team_a) & (ext.season == season)
        ext.loc[a_filter, 'points_a'] = ext[a_filter].apply(lambda row : 3 if row.res == 'HOME' else (1  if row.res == 'DRAW' else 0), axis = 1)
        ext.loc[a_filter, 'points_a'] = ext[a_filter]['points_a'].cumsum()

    # team_b
    for team_b in ext.team_b.unique():
        team_filter = (ext['team_b'] == team_b) & (ext.season == season)
        ext.loc[team_filter, 'points_b'] = ext[team_filter].apply(lambda row : 3 if row.res == 'AWAY' else (1  if row.res == 'DRAW' else 0), axis = 1)
        ext.loc[team_filter, 'points_b'] = ext[team_filter]['points_b'].cumsum()

func_points(ext, season)

# verification
team_a = 'Rennes'
ext[((ext.team_a == 'Rennes')) & (ext.season == season)][['res','points_a', 'points_b']].head()


,res,points_a,points_b
3548,DRAW,1,1
3529,DRAW,2,1
3509,HOME,5,0
3471,HOME,8,7
3459,AWAY,8,7


In [144]:
# lab
ext = raw_ext.copy()

#Cration de res : resultat du match
def func_res(row):
    res = 'HOME'
    if(row.score_b > row.score_a):
        res = 'AWAY'
    elif(row.score_b == row.score_a):
        res = 'DRAW'
    return res

ext['res'] = ext.apply(func_res, axis=1)
print(ext.shape)
ext.head(5)
# lab

season = "2010/2011"
def func_points(ext, season):

    for team_a in ext.team_a.unique():
        #print(team_a)
        a_filter_home = (ext['team_a'] == team_a) & (ext.season == season)
        ext.loc[a_filter_home, 'points_a_home'] = ext[a_filter_home]['points_a'].cumsum()     

        a_filter_away = (ext['team_b'] == team_a) & (ext.season == season)
        ext.loc[a_filter_away, 'points_a_away'] = ext[a_filter_away]['points_b'].cumsum()
        

#func_points(ext, season)
team_a = 'Rennes'
ext['points_a'] = ext.apply(lambda row : 3 if row.res == 'HOME' else (1  if row.res == 'DRAW' else 0), axis = 1)
ext['points_b'] = ext.apply(lambda row : 3 if row.res == 'AWAY' else (1  if row.res == 'DRAW' else 0), axis = 1)


#print(team_a)
for team_a in ext.team_a.unique():
    a_filter_home = (ext['team_a'] == team_a) & (ext.season == season)
    ext.loc[a_filter_home, 'points_a_home'] = ext[a_filter_home].apply(lambda row : 3 if row.res == 'HOME' else (1  if row.res == 'DRAW' else 0), axis = 1).cumsum()     

    a_filter_away = (ext['team_b'] == team_a) & (ext.season == season)
    ext.loc[a_filter_away, 'points_a_away'] = ext[a_filter_away].apply(lambda row : 3 if row.res == 'AWAY' else (1  if row.res == 'DRAW' else 0), axis = 1).cumsum()



# verification
team_a = 'Rennes'

a_filter_home = (ext['team_a'] == team_a) & (ext.season == season)
ext[a_filter_home][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(10)
ext.loc[a_filter_home]
#ext[a_filter_home].apply(lambda row : 3 if row.res == 'HOME' else (1  if row.res == 'DRAW' else 0), axis = 1).cumsum()     

# a_filter_away = (ext['team_b'] == team_a) & (ext.season == season)
# ext[a_filter_away][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(10)

#ext[a_filter_away].apply(lambda row : 3 if row.res == 'AWAY' else (1  if row.res == 'DRAW' else 0), axis = 1).cumsum()


#ext[((ext.team_a == team_a)) & (ext.season == season)][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(10)
# a_filter_away_home = ((ext['team_a'] == team_a) | (ext['team_b'] == team_a)) & (ext.season == season)
# ext[a_filter_away_home][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(30)
# attention un match nul 
# 3548

(5972, 12)


,date,day,hours,odds_a,odds_b,odds_draw,score_a,score_b,season,team_a,team_b,res,points_a,points_b,points_a_home,points_a_away
3548,2010-08-07 20:00:00,Saturday,20:00,2.74,2.67,3.08,1,1,2010/2011,Rennes,Lille,DRAW,1,1,1.0,1.0
3529,2010-08-21 18:00:00,Saturday,18:00,1.71,5.37,3.40,0,0,2010/2011,Rennes,St Etienne,DRAW,1,1,2.0,1.0
3509,2010-09-11 18:00:00,Saturday,18:00,1.66,5.59,3.51,2,1,2010/2011,Rennes,Sochaux,HOME,3,0,5.0,0.0
3471,2010-10-03 20:00:00,Sunday,20:00,2.08,3.79,3.08,3,1,2010/2011,Rennes,Toulouse,HOME,3,0,8.0,7.0
3459,2010-10-23 18:00:00,Saturday,18:00,1.83,4.63,3.30,0,1,2010/2011,Rennes,Montpellier,AWAY,0,3,8.0,7.0
3437,2010-11-06 20:00:00,Saturday,20:00,2.69,2.74,3.06,1,1,2010/2011,Rennes,Lyon,DRAW,1,1,9.0,5.0
3419,2010-11-20 18:00:00,Saturday,18:00,1.80,4.79,3.30,2,1,2010/2011,Rennes,Brest,HOME,3,0,12.0,12.0
3398,2010-12-04 18:00:00,Saturday,18:00,1.80,4.77,3.33,1,0,2010/2011,Rennes,Monaco,HOME,3,0,15.0,7.0
3378,2010-12-18 18:00:00,Saturday,18:00,1.77,4.87,3.38,1,0,2010/2011,Rennes,Valenciennes,HOME,3,0,18.0,7.0
3361,2011-01-15 20:00:00,Saturday,20:00,1.35,9.66,4.55,4,0,2010/2011,Rennes,Arles-Avignon,HOME,3,0,21.0,2.0


In [102]:
ext[((ext.team_a == 'Rennes')) & (ext.season == season)][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(10)

,date,team_a,team_b,res,points_a_home,points_a_away,points_a,points_b
3548,2010-08-07 20:00:00,Rennes,Lille,DRAW,1.0,1.0,1,1
3529,2010-08-21 18:00:00,Rennes,St Etienne,DRAW,2.0,1.0,1,1
3509,2010-09-11 18:00:00,Rennes,Sochaux,HOME,5.0,0.0,3,0
3471,2010-10-03 20:00:00,Rennes,Toulouse,HOME,8.0,7.0,3,0
3459,2010-10-23 18:00:00,Rennes,Montpellier,AWAY,8.0,7.0,0,3
3437,2010-11-06 20:00:00,Rennes,Lyon,DRAW,9.0,5.0,1,1
3419,2010-11-20 18:00:00,Rennes,Brest,HOME,12.0,12.0,3,0
3398,2010-12-04 18:00:00,Rennes,Monaco,HOME,15.0,7.0,3,0
3378,2010-12-18 18:00:00,Rennes,Valenciennes,HOME,18.0,7.0,3,0
3361,2011-01-15 20:00:00,Rennes,Arles-Avignon,HOME,21.0,2.0,3,0


In [100]:
ext[((ext.team_b == 'Rennes')) & (ext.season == season)][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a','points_b']].head(10)


,date,team_a,team_b,res,points_a_home,points_a_away,points_a,points_b
3538,2010-08-14 18:00:00,Nancy,Rennes,AWAY,0.0,3.0,0,3
3516,2010-08-28 18:00:00,Arles-Avignon,Rennes,AWAY,0.0,6.0,0,3
3493,2010-09-19 16:00:00,Paris SG,Rennes,DRAW,7.0,7.0,1,1
3490,2010-09-25 18:00:00,Nice,Rennes,AWAY,5.0,10.0,0,3
3462,2010-10-17 16:00:00,Lens,Rennes,DRAW,2.0,11.0,1,1
3424,2010-11-14 16:00:00,Auxerre,Rennes,HOME,10.0,11.0,3,0
3408,2010-11-27 18:00:00,Lorient,Rennes,HOME,16.0,11.0,3,0
3401,2010-12-01 18:00:00,Marseille,Rennes,DRAW,15.0,12.0,1,1
3381,2010-12-12 20:00:00,Bordeaux,Rennes,DRAW,13.0,13.0,1,1
3365,2010-12-22 18:00:00,Caen,Rennes,HOME,9.0,13.0,3,0


In [53]:
a_filter_away_home = ((ext['team_a'] == team_a) | (ext['team_b'] == team_a)) & (ext.season == season)
ext[a_filter_away_home][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a']].head(10)


,date,team_a,team_b,res,points_a_home,points_a_away,points_a
3548,2010-08-07 20:00:00,Rennes,Lille,DRAW,1.0,1.0,NaN
3538,2010-08-14 18:00:00,Nancy,Rennes,AWAY,NaN,3.0,NaN
3529,2010-08-21 18:00:00,Rennes,St Etienne,DRAW,2.0,1.0,NaN
3516,2010-08-28 18:00:00,Arles-Avignon,Rennes,AWAY,NaN,6.0,NaN
3509,2010-09-11 18:00:00,Rennes,Sochaux,HOME,NaN,0.0,NaN
3493,2010-09-19 16:00:00,Paris SG,Rennes,DRAW,NaN,7.0,NaN
3490,2010-09-25 18:00:00,Nice,Rennes,AWAY,NaN,10.0,NaN
3471,2010-10-03 20:00:00,Rennes,Toulouse,HOME,NaN,7.0,NaN
3462,2010-10-17 16:00:00,Lens,Rennes,DRAW,1.0,11.0,NaN
3459,2010-10-23 18:00:00,Rennes,Montpellier,AWAY,NaN,7.0,NaN


In [40]:
ext[(ext.team_b == 'Rennes') & (ext.season == season)][['date','team_a', 'team_b','res','points_a_home','points_a_away', 'points_a']].head(10)

,date,team_a,team_b,res,points_a_home,points_a_away,points_a
3538,2010-08-14 18:00:00,Nancy,Rennes,AWAY,NaN,3.0,0
3516,2010-08-28 18:00:00,Arles-Avignon,Rennes,AWAY,NaN,6.0,0
3493,2010-09-19 16:00:00,Paris SG,Rennes,DRAW,NaN,7.0,1
3490,2010-09-25 18:00:00,Nice,Rennes,AWAY,NaN,10.0,0
3462,2010-10-17 16:00:00,Lens,Rennes,DRAW,1.0,11.0,1
3424,2010-11-14 16:00:00,Auxerre,Rennes,HOME,10.0,11.0,3
3408,2010-11-27 18:00:00,Lorient,Rennes,HOME,NaN,11.0,3
3401,2010-12-01 18:00:00,Marseille,Rennes,DRAW,NaN,12.0,1
3381,2010-12-12 20:00:00,Bordeaux,Rennes,DRAW,NaN,13.0,1
3365,2010-12-22 18:00:00,Caen,Rennes,HOME,NaN,13.0,3


## etat_form_a et b 
Somme des victoires sur les 5 derniers matchs

A affiner par saison ? 

In [ ]:
season = "2010/2011"
def func_etat_form(ext, season):
    # 0 - loss, 0,5 - draw, 1 win
    ext['etat_form_a'] = 0
    ext['etat_form_b'] = 0

    # si rencontre victoire pour A etat_form_a_counter =  1
    ext['etat_form_a'] = ext.apply(lambda row : 1 if row.res == 'HOME' else (0.5  if row.res == 'DRAW' else 0), axis = 1)
    ext['etat_form_b'] = ext.apply(lambda row : 1 if row.res == 'AWAY' else (0.5  if row.res == 'DRAW' else 0), axis = 1)

    # on fait la somme glissante sur les 5 dernier match pour la team en question
    for team_a in ext.team_a.unique():
        data_filter = (ext['team_a'] == team_a) & (ext.season == season)
        ext.loc[data_filter, 'etat_form_a'] = ext[data_filter]['etat_form_a'].rolling(5).sum()

    for team_b in ext.team_b.unique():
        data_filter = (ext['team_b'] == team_b) & (ext.season == season)
        ext.loc[data_filter, 'etat_form_b']  = ext[data_filter]['etat_form_b'].rolling(5).sum()

    # on met à  0 les Nan du rolling
    ext['etat_form_a'] = ext['etat_form_a'].fillna(0)
    ext['etat_form_b'] = ext['etat_form_b'].fillna(0)

func_etat_form(ext, season)

# verification
team_a = 'Rennes'
ext[(ext.team_a == 'Rennes') & (ext.season == season)][['etat_form_a','res', 'etat_form_b']].head(10)

## but_pour_a et b, but_contre_a et b, but_diff_a et b 
Cela pourrait etre interessant de le fair pour deux équipes
- A affiner par saison


In [ ]:
# total des but mis et pris pendant 
 
ext['but_pour_a'] = ext.score_a.cumsum()
ext['but_contre_a'] = ext.score_b.cumsum()
ext['but_pour_b'] = ext.score_b.cumsum()
ext['but_contre_b'] = ext.score_a.cumsum()

# diffrence de but (but_pour - but_contre)
ext['but_diff_a'] = ext['but_pour_a'] - ext['but_contre_a']
ext['but_diff_b'] = ext['but_pour_b'] - ext['but_contre_b']

ext[['but_pour_a', 'score_a','but_contre_a', 'score_b', 'but_diff_a']].head(5)

## joue_a et b 
Nombre de match joué par les équipes 

In [ ]:
# match nombre de joué pour les teams
ext['joue_a'] = 1
ext['joue_b'] = 1

for team_a in ext.team_a.unique():
    ext.loc[ext['team_a'] == team_a, 'joue_a'] = ext[ext.team_a == team_a]['joue_a'].cumsum()

for team_b in ext.team_b.unique():
    ext.loc[ext['team_b'] == team_b, 'joue_b'] = ext[ext.team_b == team_b]['joue_b'].cumsum()

## win_a et b 
nombre de victoire des équipes 

In [ ]:
# trop corr au classement ? 
ext['win_a'] = 0
ext['win_b'] = 0

ext['win_a'] = ext['res'].apply(lambda res : 1 if (res == 'HOME') else 0)
ext['win_b'] = ext['res'].apply(lambda res : 1 if (res == 'AWAY') else 0)

for team_a in ext.team_a.unique():
    ext.loc[ext['team_a'] == team_a, 'win_a'] = ext[ext.team_a == team_a]['win_a'].cumsum()

for team_b in ext.team_b.unique():
    ext.loc[ext['team_b'] == team_b, 'win_b'] = ext[ext.team_b == team_b]['win_b'].cumsum()

# verification 
team_a = 'Rennes'
ext[ext.team_b == team_a][['team_a','res', 'team_b', 'win_b']].head()

## loss_a et b 

In [ ]:
ext['loss_a'] = 0
ext['loss_b'] = 0

ext['loss_a'] = ext['res'].apply(lambda res : 1 if (res == 'AWAY') else 0)
ext['loss_b'] = ext['res'].apply(lambda res : 1 if (res == 'HOME') else 0)

for team_a in ext.team_a.unique():
    ext.loc[ext['team_a'] == team_a, 'loss_a'] = ext[ext.team_a == team_a]['loss_a'].cumsum()

for team_b in ext.team_b.unique():
    ext.loc[ext['team_b'] == team_b, 'loss_b'] = ext[ext.team_b == team_b]['loss_b'].cumsum()

# verification 
team_a = 'Rennes'
ext[ext.team_a == team_a][['team_a','res', 'team_b', 'loss_a']].head()

## draw_a et b 

In [ ]:
ext['draw_a'] = 0
ext['draw_b'] = 0

ext['draw_a'] = ext['res'].apply(lambda res : 1 if (res == 'DRAW') else 0)
ext['draw_b'] = ext['res'].apply(lambda res : 1 if (res == 'DRAW') else 0)

for team_a in ext.team_a.unique():
    ext.loc[ext['team_a'] == team_a, 'draw_a'] = ext[ext.team_a == team_a]['draw_a'].cumsum()

for team_b in ext.team_b.unique():
    ext.loc[ext['team_b'] == team_b, 'draw_b'] = ext[ext.team_b == team_b]['draw_b'].cumsum()

## Autre features : 
- Pour une rencontre, historique des confrontations pour les deux equipes ? 
- Gerer les dates ? 
- Par season ! 

On décale d'une date les variables qui ont été calculé à partir du score (qu'on ne connait pas au moment de la rencontre.)
On revanche on connait la date, l'heure, les deux teams et les cotes. 

In [ ]:
['date', 'day', 'hours', 'odds_a', 'odds_b', 'odds_draw', 'score_a',
       'score_b', 'season', 'team_a', 'team_b', 'res', 'min_odds', 'max_odds',
       'is_booky_write', 'worst_pred', 'draw_pred', 'points_a', 'points_b',
       'etat_form_a', 'etat_form_b', 'but_pour_a', 'but_contre_a',
       'but_pour_b', 'but_contre_b', 'but_diff_a', 'but_diff_b', 'joue_a',
       'joue_b', 'win_a', 'win_b', 'loss_a', 'loss_b', 'draw_a', 'draw_b']
feature_to_shift = ['score_a','score_b','res', 'is_booky_write', 'worst_pred', 'draw_pred', 'points_a', 'points_b',
       'etat_form_a', 'etat_form_b', 'but_pour_a', 'but_contre_a',
       'but_pour_b', 'but_contre_b', 'but_diff_a', 'but_diff_b', 
                    'win_a', 'win_b', 'loss_a', 'loss_b', 'draw_a', 'draw_b']
print()

In [ ]:
ext[ext.team_a == 'Rennes']

In [ ]:
# on veut prédire la prochaine rencontre : 
# on shift donc les données par team 
data = ext.copy()

# je tout poru une team, mais shifter les colones de b n'a pas de sens
# il faut faire la diff entre variables du match : res ou is_booky_write (non shiftable mais non utilisable pour la pred)
# et les variables de team_a et b qui donne de l'info sur les perf de la team
shift_a = ['score_a','points_a',
       'etat_form_a','but_pour_a', 'but_contre_a',
       'but_diff_a', 'win_a', 'loss_a', 'draw_a']

for team_a in data.team_a.unique():
    data.loc[data['team_a'] == team_a, shift_a] = data[data['team_a'] == team_a][shift_a].shift(periods=1) 

shift_b = ['score_b','points_b',
       'etat_form_b','but_pour_b', 'but_contre_b',
       'but_diff_b', 'win_b', 'loss_b', 'draw_b']

for team_b in data.team_b.unique():
    data.loc[data['team_b'] == team_b, shift_b] = data[data['team_b'] == team_b][shift_b].shift(periods=1) 

In [ ]:
# verification - en fessant ca on a inclu plein de NaN
data[data['team_a'] == 'Nice'].head(1)
# Il faut absolument le faire pair saison et mettre tout un pipeline
data.isna().sum()

In [ ]:
# Du coup on prédit quoi ?

In [ ]:


file_name = "exctract_enrichies"
thefile = os.path.join(package_dir, file_name+'.csv')
ext.to_csv(thefile,index=False)